<a href="https://colab.research.google.com/github/amha-kindu/neural-network-coding/blob/main/Amha_Kindu_Lab_4_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch

In [52]:
class DenseLayer:
  activations = {
      "relu": lambda x: torch.max(torch.tensor(0, dtype=x.dtype), x),
      "softmax": lambda x, dim=1: torch.exp(x - torch.max(x, dim=dim, keepdim=True).values) / torch.exp(x - torch.max(x, dim=dim, keepdim=True).values).sum(dim=dim, keepdim=True),
      "sigmoid": lambda x: 1 / (1 + torch.exp(-x))
  }
  def __init__(self, features=1, neurons=1, activation="relu"):
    self.weights = torch.rand(features, neurons)
    self.bias = torch.rand((1, neurons))
    self.activation = activation

  def forward(self, inputs):
    activation = DenseLayer.activations[self.activation]
    return activation(torch.matmul(inputs, self.weights) + self.bias)

In [53]:
class NeuralNetwork:
    def __init__(self, layers: list[DenseLayer]):
        self.layers = layers

    def forward(self, inputs):
        output = inputs
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def mse_loss(self, prediction, target):
        return torch.mean((prediction - target)**2)

    def backward(self, inputs, targets, learning_rate=0.01):
        predictions = self.forward(inputs)

        loss = self.mse_loss(predictions, targets)

        loss_grad = 2 * (predictions - targets) / len(targets)  # Gradient of MSE loss

        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]

            layer_grad = torch.matmul(inputs, loss_grad)
            bias_grad = torch.sum(loss_grad, dim=0, keepdim=True)

            layer.linear.weight -= learning_rate * layer_grad
            layer.linear.bias -= learning_rate * bias_grad

            loss_grad = torch.matmul(loss_grad, layer.linear.weight.t())

    def train(self, inputs, targets, learning_rate=0.01, num_epochs=1000):
        for epoch in range(num_epochs):
            self.backward(inputs, targets, learning_rate)
            if (epoch + 1) % 100 == 0:
                predictions = self.forward(inputs)
                loss = self.mse_loss(predictions, targets)
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


    def cross_entropy_loss(self, prediction, target):
      return -torch.sum(prediction * torch.log(target))

    def accuracy(self, prediction, target):
      _, predicted_classes = torch.max(prediction, 1)
      _, true_classes = torch.max(target, 1)

      correct_predictions = (predicted_classes == true_classes).sum().item()
      return correct_predictions / target.size(0)

In [54]:
model = NeuralNetwork([
    DenseLayer(4, 18, "relu"),
    DenseLayer(18, 18, "sigmoid"),
    DenseLayer(18, 18, "relu"),
    DenseLayer(18, 1, "softmax")
])
model.layers

In [55]:
x = torch.rand(1, 4)
target = torch.rand(1, 3)
print(f"Input: {x}\nTarget: {target}")

Input: tensor([[0.0149, 0.0180, 0.1160, 0.9163]])
Target: tensor([[0.6785, 0.0093, 0.8334]])


In [56]:
prediction = model.forward(x)
prediction

tensor([[1.]])

In [57]:
loss = model.cross_entropy_loss(prediction, target)
accuracy = model.accuracy(prediction, target)
print(f"Cross-entropy loss: {loss}\nAccuracy: {accuracy}")

Cross-entropy loss: 5.246840000152588
Accuracy: 0.0


In [58]:
# Test Input (x)
test_input = torch.tensor([1, 2, 3, 4], dtype=torch.float32)

# Expected Output (f(x))
test_output = test_input.pow(2) + 3 * test_input

In [59]:
model.train(test_input, test_output, learning_rate=0.01, num_epochs=1000)

RuntimeError: ignored